Feedrank and category on Viettel subjects
=====================

To rank a feed is fact or not?

* Unsupervised to cluster on Kmean based
* Categorized based on auto label from unsuperived clusters

In [1]:
import re 
import string
import unicodedata
import sys

STOPWORDS = [u"ấy", u"bị", u"bởi", u"cả", u"các", u"cái", u"cần", u"càng", u"chỉ", u"chiếc", u"cho", u"chứ", u"chưa", 
             u"chuyện", u"có", u"có_thể", u"cứ", u"của", u"cùng", u"cũng", u"đã", u"đang", u"đây", u"để", u"đến_nỗi", 
             u"đều", u"điều", u"do", u"đó", u"được", u"dưới", u"gì", u"khi", u"không", u"là", u"lại", u"lên", u"lúc", 
             u"mà", u"mỗi", u"một cách", u"này", u"nên", u"nếu", u"ngay", u"nhiều", u"như", u"nhưng", u"những", u"nơi", 
             u"nữa", u"phải", u"qua", u"ra", u"rằng", u"rất", u"rồi", u"sau", u"sẽ", u"so", u"sự", u"tại", u"theo", 
             u"thì", u"trên", u"trước", u"từ", u"từng", u"và", u"vẫn", u"vào", u"vậy", u"vì", u"việc", u"với", u"vừa",
             u"_num", u"wwdateww", u"wwtimeww", u"wwemailww", u"wwipww", u"wwurlww", u"wwnumberww"
            ]

tbl = dict.fromkeys(i for i in xrange(sys.maxunicode)
                      if unicodedata.category(unichr(i)).startswith('P') and i != 45 and i!= 95)

def vi_trans_unicode(su):
    return su.translate(tbl)

def vi_strip_text2(s):
    s = re.sub(r"&amp;", "", s)
    s = vi_trans_unicode(s)
    s = re.sub(r"<([^>]+)>", "", s)
    s = re.sub(r"(\s|\\n|\\r|\\t)+", " ", s)
    s = re.sub(r"__+", "_", s)
    s = re.sub(r"--+", "-", s)
    s = re.sub(r'(.)\1+', r'\1\1', s)
    s = ' '.join([w if not w[0].isdigit() else u"wwNUMBERww" for w in s.strip().split()])
    return s


def vi_clean3(line):
    words = line.replace('.','').strip().split()
    words = [w.lower() for w in words]
    return ' '.join(words)

def vi_remove_stop_1char(line):
    words = line.split()
    words = [w for w in words if w not in STOPWORDS and len(w) > 1]
    return ' '.join(words)

import requests
import codecs
import json

tok_url = "http://192.168.0.215:8081/api/v1.0/document/filter"

def get_tokens(tok_url, data):
    try:
        rq = requests.post(tok_url, data=data.encode('utf-8'))
        if rq.content:
            tok_doc = ' '.join(json.loads(rq.content)['sentences'])
        else:
            tok_doc = None
        return tok_doc
    except Exception, e:
        print e

In [2]:
from sklearn.externals import joblib

## Unsupervised

In [4]:
vectorizer_tfidf = joblib.load("/home/laampt/nlp/feedrank/model/feedrank_24Kdoc_vectorizer_tfidf.pkl")
mbk20 = joblib.load("/home/laampt/nlp/feedrank/model/feedrank_24Kdoc_mbk20.pkl")
mbk50 = joblib.load("/home/laampt/nlp/feedrank/model/feedrank_24Kdoc_mbk50.pkl")

In [5]:
print vectorizer_tfidf

TfidfVectorizer(analyzer='word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.95, max_features=None, min_df=2,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=True,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)


In [6]:
print mbk20
print mbk50

MiniBatchKMeans(batch_size=2000, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=100,
        n_clusters=20, n_init=10, random_state=None,
        reassignment_ratio=0.01, tol=0.0, verbose=0)
MiniBatchKMeans(batch_size=2000, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=100,
        n_clusters=50, n_init=10, random_state=None,
        reassignment_ratio=0.01, tol=0.0, verbose=0)


In [7]:
def mbk_prediction(feed, vectorizer_tfidf, mbk2, mbk50):
    """
    Input: given raw feed, vectorizer, kmean 20 clusters, kmean 50 clusters
    Output: 1 if feed is user opinion alike else 0
    """
    s_test_clean = vi_remove_stop_1char(vi_clean3(vi_strip_text2(feed)))
    vecs = vectorizer_tfidf.transform([s_test_clean])
    p20 = mbk2.predict(vecs)[0]
    p50 = mbk50.predict(vecs)[0]
    
    if (p20 == 10 and p50 == 35) or (p20 == 11 and p50 == 31) or (p20 == 12 and p50 == 32) or (p20 == 16 and p50 == 9):
        prediction = 1
    else:
        prediction = 0
    return prediction

In [8]:
feed = u"minh gooner chuyển xài viettel gói_cước đi thấy xài good nhá"
print mbk_prediction(feed, vectorizer_tfidf, mbk20, mbk50)

1


## Supervised

In [20]:
lr_tfidf10 = joblib.load("/home/laampt/nlp/feedrank/estimator/39/lr_tfidf_99auc_10cv2.pkl")
svc_tfidf10 = joblib.load("/home/laampt/nlp/feedrank/estimator/39/svc_tfidf_99auc_10cv2.pkl")

In [21]:
print lr_tfidf10
print svc_tfidf10

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)
LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)


In [28]:
def mega_prediction(feed, vectorizer_tfidf, lr_tfidf10, svc_tfidf10, w=0.6):
    """
    Input: given raw feed, vectorizer, lr, svm classifier
    Output: 1 if feed is user opinion alike else 0
    """
    s_test_clean = vi_remove_stop_1char(vi_clean3(vi_strip_text2(feed)))
    vec = vectorizer_tfidf.transform([s_test_clean]).toarray()[0,:].T
    prediction = lr_tfidf10.predict(vec) * w + svc_tfidf10.predict(vec) * (1-w)
    prediction = 1 if (prediction[0] > 0.5) else 0
    return prediction

In [29]:
feed = u"minh gooner chuyển xài viettel gói_cước đi thấy xài good nhá"
print mega_prediction(feed, vectorizer_tfidf, lr_tfidf10, svc_tfidf10)

1


## Sanity check

In [42]:
pos1 = u"viettel làm_việc chuyên_nghiệp và mình tới đây lần nào cũng hài_lòng"
neg2 = u"chương_trình khuyến_mãi thẻ_cào nhân dịp quốc_khánh wwDATEww Nạp wwNUMBERww được wwNUMBERww ông chú vt bảo vậy"

pred_mbk_pos1 = mbk_prediction(pos1, vectorizer_tfidf, mbk20, mbk50)
pred_mbk_neg2 = mbk_prediction(neg2, vectorizer_tfidf, mbk20, mbk50)

pred_mega2_pos1 = mega_prediction(pos1, vectorizer_tfidf, lr_tfidf10, svc_tfidf10)
pred_mega2_neg2 = mega_prediction(neg2, vectorizer_tfidf, lr_tfidf10, svc_tfidf10)

print "REPORT"
print "-" * 80

print "POS: "
print pos1
print "PREDICT: MBK [{}] MEGA [{}]".format(pred_mbk_pos1, pred_mega2_pos1)
print "." * 80

print "NEG: "
print neg2
print "PREDICT: MBK [{}] MEGA [{}]".format(pred_mbk_neg2, pred_mega2_neg2)
print "." * 80

REPORT
--------------------------------------------------------------------------------
POS: 
viettel làm_việc chuyên_nghiệp và mình tới đây lần nào cũng hài_lòng
PREDICT: MBK [0] MEGA [0]
................................................................................
NEG: 
chương_trình khuyến_mãi thẻ_cào nhân dịp quốc_khánh wwDATEww Nạp wwNUMBERww được wwNUMBERww ông chú vt bảo vậy
PREDICT: MBK [0] MEGA [0]
................................................................................


In [34]:
inp = u"tôi yêu viettel làm việc ngon lành luôn nhiệt tình nguyện hết mình"
feed = get_tokens(tok_url, inp)
print feed
print "PREDICT: MBK {}  MEGA {}".format(mbk_prediction(feed, vectorizer_tfidf, mbk20, mbk50), mega_prediction(feed, vectorizer_tfidf, lr_tfidf10, svc_tfidf10))

tôi yêu viettel làm_việc ngon_lành luôn nhiệt tình_nguyện hết_mình
PREDICT: MBK 0  MEGA 1


## Benchmark

In [35]:
%load_ext memory_profiler

In [38]:
%timeit mbk_prediction(feed, vectorizer_tfidf, mbk20, mbk50)

100 loops, best of 3: 1.8 ms per loop


In [39]:
%timeit mega_prediction(feed, vectorizer_tfidf, lr_tfidf10, svc_tfidf10)

The slowest run took 4.63 times longer than the fastest. This could mean that an intermediate result is being cached 
1000 loops, best of 3: 837 µs per loop


In [40]:
%memit mbk_prediction(feed, vectorizer_tfidf, mbk20, mbk50)

peak memory: 64.14 MiB, increment: 0.18 MiB


In [41]:
%memit mega_prediction(feed, vectorizer_tfidf, lr_tfidf10, svc_tfidf10)

peak memory: 64.18 MiB, increment: 0.04 MiB
